# Run PheWAS using PheWAS package

## As-Is Software Disclaimer

This notebook is delivered "As-Is". Notwithstanding anything to the contrary, DNAnexus will have no warranty, support, liability or other obligations with respect to Materials provided hereunder.

[MIT License](https://github.com/dnanexus/UKB_RAP/blob/main/LICENSE) applies to this notebook.

## Jupyterlab app details (launch configuration)

Recommended configuration
- Runtime: ~ 6 hours
- Cluster configuration: `Single Node`
- Recommended instance: `mem2_ssd1_v2_x32`
- Cost:  ~£5.5

## Dependencies
|Library |License|
|:------------- |:-------------|
|[cli](https://cli.r-lib.org/) | [MIT](https://cli.r-lib.org/LICENSE.html) |
|[devtools](https://devtools.r-lib.org/) | [MIT](https://devtools.r-lib.org/LICENSE.html) |
|[dplyr](https://dplyr.tidyverse.org) | [MIT](https://dplyr.tidyverse.org/LICENSE.html) |
|[DT](https://rstudio.github.io/DT/) | [GPL-3](https://github.com/rstudio/DT/blob/main/LICENSE) |
|[ggplot2](https://ggplot2.tidyverse.org) | [MIT](https://ggplot2.tidyverse.org/LICENSE.html) |
|[ggrepel](https://ggrepel.slowkow.com/articles/examples.html) | [GPL-3](https://github.com/slowkow/ggrepel/blob/master/LICENSE) |
|[MASS](https://cran.r-project.org/web/packages/MASS/index.html) | [GPL-3](https://cran.r-project.org/web/licenses/GPL-3) |
|[meta](https://cran.r-project.org/web/packages/meta/index.html) | [GPL-3](https://cran.r-project.org/web/licenses/GPL-3) |
|[PheWAS](https://github.com/PheWAS/PheWAS) | [GPL-3](https://www.gnu.org/licenses/gpl-3.0.en.html)|
|[tidyr](https://tidyr.tidyverse.org) | [MIT](https://tidyr.tidyverse.org/LICENSE.html) |
|[tidyverse](https://www.tidyverse.org/) |[MIT](https://github.com/tidyverse/tidyverse/blob/main/LICENSE.md) |
|[vctrs](https://vctrs.r-lib.org/) | [MIT](https://vctrs.r-lib.org/LICENSE.html) |


## Introduction

This notebook shows how to run PheWAS analysis using [PheWAS R package](https://github.com/PheWAS/PheWAS).

Data used for this notebook:
- Phenome data in long format containing ICD10 codes per participant (`pheno_icd10_long.csv`)
- Table containing covariates (`ischemia.covariate`)
- Genetic data for each rsID (`<rsid>.csv`)


## Preparing your environment

Uncomment the install commands if you are comfortable with the library license and want to install and run the parts notebook that depend on the library.

In [ ]:
system('apt update')
system('apt -y upgrade')
system('apt -y install libnlopt-dev')
system('apt -y install libcurl4-openssl-dev')
system('apt -y install libbz2-dev liblzma-dev')
system('apt -y install libfontconfig1-dev')
system('apt -y install libharfbuzz-dev libfribidi-dev')
system('apt -y install libfreetype6-dev libpng-dev libtiff5-dev libjpeg-dev')
system('apt -y install cmake')

In [ ]:
#install.packages('devtools')
#install.packages('cli')
#install.packages(c('dplyr','tidyr','ggplot2','MASS','meta','ggrepel','DT'))
#devtools::install_github('PheWAS/PheWAS')

**You may need to restart kernel for the changes to apply after running the following code.**

In [ ]:
#remove.packages('vctrs')
#remove.packages('dplyr')

#install.packages('vctrs')
#install.packages('dplyr')

In [ ]:
library(ggrepel)
library(PheWAS)

## Load phenotype and covariate data

In [ ]:
system('dx download /Data/PheWAS/pheno_icd10_long.csv')

In [ ]:
system('dx download /Data/PheWAS/covariates.txt')

In [ ]:
pheno = read.csv('./pheno_icd10_long.csv')
head(pheno)

In [ ]:
covariate = read.csv('covariates.txt', sep = '\t')
head(covariate)

In [ ]:
# Map ICD10 codes to code description
phenotypes = createPhenotypes(
    pheno, min.code.count=1, add.phecode.exclusions=T, translate=T, vocabulary.map=PheWAS::phecode_map_icd10
)

## Run PheWAS

In [ ]:
rsid_files = list.files(path='/mnt/project/Data/phewas_geno_data', pattern=glob2rx('rs*txt'))
rsid_files[1:10]

In [ ]:
detectCores()

In [ ]:
# Create table to aggregate significant results from all variants
sig_phewas = data.frame()

# Create directories to write PheWAS results and plots for each variant
ifelse(!dir.exists('png'), dir.create('png'), FALSE)
ifelse(!dir.exists('csv'), dir.create('csv'), FALSE)

# Run PheWAS for each variant separately
for (file in rsid_files){
    rsid = strsplit(file, '.txt')[[1]]
    print(rsid)
    # Load geno data
    geno_data = read.csv(sprintf('/mnt/project/Data/phewas_geno_data/%s', file), sep = '\t')
    results = phewas(
    phenotypes, geno_data, cores = detectCores(), significance.threshold = c('p-value', 'bonferroni', 'fdr'),
    covariates = covariate, additive.genotypes = FALSE)

    # Add PheWAS descriptions
    results_d=addPhecodeInfo(results)
    
    # Get significant results
    res = results_d[results_d$bonferroni & !is.na(results_d$p),]
    #print("res")
    #print(res)
    print("sig_phewas")
    sig_phewas = rbind(sig_phewas, res)
    print(sig_phewas)
    
    # Re-create the same threshold for plots as is used in bonferroni column
    sig_p = 0.05/(nrow(results_d[!is.na(results_d$p),]))
    print("sig_p")
    print(sig_p)
    print(nrow(results_d[!is.na(results_d$p),]))
    
    # Exctract significant result and save it as csv
    results_d = results_d[!is.na(results_d$p),]
    results_d = results_d[order(results_d$group),]
    results_d$order_num = rep(1:nrow(results_d))
    results_d$order_num = factor(results_d$order_num, levels = results_d$order_num)
    write.csv(results_d, sprintf('csv/%s_phewas.csv', rsid), row.names = FALSE)
    
    # Create Manhattan plot annotating significant phenotypes
    # Significant phenotypes are defined by passing Bonferroni correction
    png(filename=sprintf('png/%s_phewas_man.png', rsid), , width = 1400, height = 800)
    options(ggrepel.max.overlaps = Inf) 
    man_plot <- ggplot(
        results_d, 
        aes(x=order_num, y=-log(p))) + geom_point(aes(col=group, size=OR)) + theme_classic() + theme(
                axis.text.x = element_blank(), panel.grid.minor=element_line(colour = 'grey', linetype = 'dashed'), 
                axis.ticks=element_blank()
            ) + labs(color = 'Category', size = 'Effect size', x = rsid, y = '-log(p-value)') + geom_text_repel(
            data=. %>% mutate(label = ifelse((p < sig_p) & (bonferroni == TRUE), as.character(description), '')), aes(label = label), size = 3,
            box.padding = unit(0.7, 'lines')
        ) + geom_hline(yintercept=-log(sig_p), color='red', linewidth=1, alpha=0.5) 
    print(man_plot)
    dev.off()
}

Upload CSV file and PNG files to UKB RAP.

In [ ]:
system('dx upload -r csv/ --path /Data/PheWAS/csv/')

In [ ]:
system('dx upload -r png/ --path /Data/PheWAS/png/')

Save significant results as CSV and upload to UKB RAP.

In [ ]:
sig_phewas

In [ ]:
write.csv(sig_phewas, 'significant_phewas.csv', row.names = FALSE)

In [ ]:
system('dx upload significant_phewas.csv --path /Data/PheWAS/')

Aggregate significant results by phenotype, count occurrence save them as CSV and upload to UKB RAP.

In [ ]:
sig_phewas_tb <- tibble::tibble(sig_phewas)
sig_phewas_agg <- dplyr::count(sig_phewas_tb, description, sort = TRUE)
sig_phewas_agg

In [ ]:
write.csv(sig_phewas_agg, 'significant_phewas_agg.csv', row.names = FALSE)

In [ ]:
system('dx upload significant_phewas_agg.csv --path /Data/PheWAS/')

## Output files

- Results of PheWAS analysis including phenotype description for each variant (`<rsid>_phewas.csv`)
- Manhattan plot for PheWAS analysis for each variant (`<rsid>_phewas_man.png`)
- Combined significant results for all variants (`significant_phewas.csv`)
- Significant results aggregated by phenotype (`significant_phewas_agg.csv`)